### 对分类模型进行量化，并测试精度

In [1]:
import torchvision
from ppq import *
from ppq.api import *
from Utilities.Imagenet import (evaluate_mmlab_module_with_imagenet,
                                evaluate_onnx_module_with_imagenet,
                                evaluate_ppq_module_with_imagenet,
                                evaluate_torch_module_with_imagenet,
                                load_imagenet_from_directory)
import os


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/


TensorRT is not installed, TRT Exporter is disabled.


### 测试全精度模型

In [4]:
CFG_PLATFORM = TargetPlatform.OPENVINO_INT8   # 用来指定目标平台
platform = "OpenVino"

CFG_DEVICE = 'cuda'                            # 一个神奇的字符串，用来确定执行设备
CFG_BATCHSIZE = 64                             # 测试与calib时的 batchsize
CFG_INPUT_SHAPE = (CFG_BATCHSIZE, 3, 224, 224) # 用来确定模型输入的尺寸，好像 imagenet 都是这个尺寸
CFG_VALIDATION_DIR = '/home/geng/tinyml/ppq/benchmark/Assets/Imagenet_Valid'   # 用来读取 validation dataset
CFG_TRAIN_DIR = '/home/geng/tinyml/ppq/benchmark/Assets/Imagenet_Train'        # 用来读取 train dataset，注意该集合将被用来 calibrate 你的模型
CFG_DUMP_PATH = '/home/geng/tinyml/ppq/benchmark/classification/'+platform+'_output'    # 所有模型保存的路径名
QUANT_SETTING = QuantizationSettingFactory.default_setting() # 用来指定量化配置
if not os.path.exists(CFG_DUMP_PATH):
    os.makedirs(CFG_DUMP_PATH)

|model|TargetPlatform|ORT FP32|PPQ INT8|DQD ORT INT8|RealPlatform INT8|
|----|----|----|----|----|----|
|resnet18|OpenVino|69.764|69.466|67.109|-|

In [5]:
with ENABLE_CUDA_KERNEL():
    model_builder, model_name  = torchvision.models.resnet18, 'resnet18'

    print(f'---------------------- PPQ Quantization Test Running with {model_name} ----------------------')
    model = model_builder(pretrained=True).to(CFG_DEVICE)

    #测试FP32精度
    fp32_report = evaluate_torch_module_with_imagenet(
        model=model, imagenet_validation_dir=CFG_VALIDATION_DIR,
        batchsize=CFG_BATCHSIZE, device=CFG_DEVICE, verbose=True)

    # 获取校准数据
    dataloader = load_imagenet_from_directory(
        directory=CFG_TRAIN_DIR, batchsize=CFG_BATCHSIZE,
        shuffle=False, subset=5120, require_label=False,
        num_of_workers=8)

    # 量化torch模型
    ppq_quant_ir = quantize_torch_model(
        model=model, calib_dataloader=dataloader, input_shape=CFG_INPUT_SHAPE,
        calib_steps=5120 // CFG_BATCHSIZE, collate_fn=lambda x: x.to(CFG_DEVICE), verbose=1,
        device=CFG_DEVICE, platform=CFG_PLATFORM, setting=QUANT_SETTING,
        onnx_export_file=f'{os.path.join(CFG_DUMP_PATH, model_name)}-FP32.onnx')
        
    # 评估PPQ量化后的模型
    ppq_int8_report = evaluate_ppq_module_with_imagenet(
        model=ppq_quant_ir, imagenet_validation_dir=CFG_VALIDATION_DIR,
        batchsize=CFG_BATCHSIZE, device=CFG_DEVICE, verbose=True)

    # 导出平台部署模型
    export_ppq_graph(
        graph=ppq_quant_ir, 
        platform=CFG_PLATFORM,
        graph_save_to=f'{os.path.join(CFG_DUMP_PATH, model_name)}-INT8.onnx',
        config_save_to=f'{os.path.join(CFG_DUMP_PATH, model_name)}-INT8.json' )
    
    # 评估onnx运行模型
    evaluate_onnx_module_with_imagenet(
        onnxruntime_model_path=f'{os.path.join(CFG_DUMP_PATH, model_name)}-INT8.onnx', 
        imagenet_validation_dir=CFG_VALIDATION_DIR, batchsize=CFG_BATCHSIZE, 
        device=CFG_DEVICE)

    # ppq_int8_report.to_csv(f'{os.path.join(CFG_DUMP_PATH, model_name)}-report.csv')


PPQ is compling CUDA Kernels. Please wait...If there is any problem with kernel compilation, feel free to remove ENABLE_CUDA_KERNEL clause.
---------------------- PPQ Quantization Test Running with resnet18 ----------------------


/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Evaluating Model...:   0%|          | 0/781 [00:00<?, ?it/s]/home/geng/tinyml/ppq/benchmark/classification/Utilities/Imagenet/imagenet_util.py:188: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prec1

Test: [0 / 781]	Prec@1 84.375 (84.375)	Prec@5 95.312 (95.312)


Evaluating Model...:  13%|█▎        | 104/781 [00:05<00:35, 18.89it/s]

Test: [100 / 781]	Prec@1 76.269 (76.269)	Prec@5 92.280 (92.280)


...

Test: [600 / 781]	Prec@1 68.342 (68.342)	Prec@5 88.150 (88.150)


Evaluating Model...:  90%|████████▉ | 701/781 [06:58<00:48,  1.65it/s]

Test: [700 / 781]	Prec@1 67.306 (67.306)	Prec@5 87.357 (87.357)


Evaluating Model...: 100%|██████████| 781/781 [07:43<00:00,  1.68it/s]

 * Prec@1 67.109 Prec@5 87.246
